In [2]:
!pip install geodatasets

In [3]:
!pip freeze > ..//requirements.txt

In [8]:
from _spark import *
from transformations import transform

import pyspark.sql.functions as f
import pyspark.sql.types as t
import os
import findspark
findspark.init()

spark = get_spark()

gcs_bucket =  'tech_challengefase-3'

In [11]:

!pip install pandas-gbq --upgrade
!pip install google-cloud-bigquery --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 3.25.0
    Uninstalling google-cloud-bigquery-3.25.0:
      Successfully uninstalled google-cloud-bigquery-3.25.0


In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
project_id = "fase3-437322"

In [ ]:
query = """
select *
from fase3-437322.Fase3dataset.microdados_consolidado
"""

In [ ]:
df = pd.read_gbq(query, project_id=project_id, dialect='standard')

In [8]:
df.head()

,UF,V1012,V1013,V1022,A002,A003,A004,A005,B0011,B0014,...,B0019,B00111,B002,B0031,B005,B009B,B007,C007B,C01011,F001
0,14,1,9,1,1,2,4,1,2,2,...,2,2,<NA>,<NA>,<NA>,<NA>,2,<NA>,<NA>,1
1,15,1,9,1,4,2,4,1,2,2,...,2,2,<NA>,<NA>,<NA>,<NA>,2,<NA>,<NA>,1
2,23,1,9,2,4,2,4,1,2,2,...,2,2,<NA>,<NA>,<NA>,<NA>,2,<NA>,<NA>,4
3,23,1,9,1,2,2,1,1,2,2,...,2,2,<NA>,<NA>,<NA>,<NA>,2,<NA>,<NA>,1
4,29,1,9,1,5,2,4,1,2,2,...,2,2,<NA>,<NA>,<NA>,<NA>,2,<NA>,<NA>,3


In [10]:
# Renomeando colunas:
renomear_colunas = {
    "UF": "uf",
    "V1012": "semana_mes",
    "V1013": "mes",
    "V1022": "area_domicilio",
    "A002": "idade",
    "A003": "sexo",
    "A004": "cor_raca",
    "A005": "escolaridade",
    "B0011": "teve_febre",
    "B0014": "teve_dificuldade_respirar",
    "B0015": "teve_dor_cabeca",
    "B0019": "teve_fadiga",
    "B00111": "teve_perda_cheiro",
    "B002": "foi_posto_saude",
    "B0031": "ficou_em_casa",
    "B005": "ficou_internado",
    "B009B": "resultado_covid",
    "B007": "tem_plano_saude",
    "C007B": "assalariado",
    "C01011": "faixa_rendimento",
    "F001": "situacao_domicilio"
}

df = df.rename(columns=renomear_colunas)

In [11]:

# Repositório de códigos:
# Nome da UF
unidade_federativa = {
    "11": "Rondônia",
    "12": "Acre",
    "13": "Amazonas",
    "14": "Roraima",
    "15": "Pará",
    "16": "Amapá",
    "17": "Tocantins",
    "21": "Maranhão",
    "22": "Piauí",
    "23": "Ceará",
    "24": "Rio Grande do Norte",
    "25": "Paraíba",
    "26": "Pernambuco",
    "27": "Alagoas",
    "28": "Sergipe",
    "29": "Bahia",
    "31": "Minas Gerais",
    "32": "Espírito Santo",
    "33": "Rio de Janeiro",
    "35": "São Paulo",
    "41": "Paraná",
    "42": "Santa Catarina",
    "43": "Rio Grande do Sul",
    "50": "Mato Grosso do Sul",
    "51": "Mato Grosso",
    "52": "Goiás",
    "53": "Distrito Federal"
}
df['uf'] = df['uf'].astype(str)
df['uf'] = df['uf'].replace(unidade_federativa)

In [12]:
# Escolaridade
escolaridade = {
        '1': 'Sem instrução',
        '2': 'Fundamental incompleto',
        '3': 'Fundamental completa',
        '4': 'Médio incompleto',
        '5': 'Médio completo',
        '6': 'Superior incompleto',
        '7': 'Superior completo',
        '8': 'Pós-graduação, mestrado ou doutorado',
    }
df['escolaridade'] = df['escolaridade'].astype(str)
df['escolaridade'] = df['escolaridade'].replace(escolaridade)

In [13]:
# area domicilio
area = {
    '1': 'Urbana',
    '2': 'Rural',
}
df['area_domicilio'] = df['area_domicilio'].astype(str)
df['area_domicilio'] = df['area_domicilio'].replace(area)

In [14]:

# sexo
sexo = {
    '1': 'Homem',
    '2': 'Mulher',
}
df['sexo'] = df['sexo'].astype(str)
df['sexo'] = df['sexo'].replace(sexo)

In [15]:

# cor-raça
cor_raca = {
    '1': 'Branca',
    '2': 'Preta',
    '3': 'Amarela',
    '4': 'Parda',
    '5': 'Indígena',
    '9': 'Ignorado',
}
df['cor_raca'] = df['cor_raca'].astype(str)
df['cor_raca'] = df['cor_raca'].replace(cor_raca)

In [18]:
# perguntas "sim/não"
sim_nao = {
    1: "Sim",
    2: "Não",
    3: "Não sabe",
    9: "Ignorado",
}
for coluna in ['teve_febre', 'teve_dor_cabeca','teve_dificuldade_respirar','teve_fadiga','teve_perda_cheiro','tem_plano_saude','foi_posto_saude','ficou_em_casa']:
    # Change the data type of the column to 'object' (string) before converting to numeric
    df[coluna] = df[coluna].astype(object)
    df[coluna] = pd.to_numeric(df[coluna], errors='coerce')  # Converte para numérico, tornando valores não numéricos em NaN
    df[coluna] = df[coluna].fillna('não aplicável')  # Substitui NaN por "não aplicável"

    # Now it's safe to replace with the dictionary
    df[coluna] = df[coluna].replace(sim_nao)

In [19]:

# Ficou internado
ficou_internado = {
    '1': 'Sim',
    '2': 'Não',
    '3': 'Não foi atendido',
    '9': 'Ignorado',
    }
df['ficou_internado'] = df['ficou_internado'].astype(str)
df['ficou_internado'] = df['ficou_internado'].replace(ficou_internado)

In [20]:

# Resultado covid
resultado_covid = {
    '1': 'Positivo',
    '2': 'Negativo',
    '3': 'Inconclusivo',
    '4': 'Ainda não recebeu o resultado',
    '9': 'Ignorado',
    }
df['resultado_covid'] = df['resultado_covid'].astype(str)
df['resultado_covid'] = df['resultado_covid'].replace(resultado_covid)


In [21]:
# Assalariado
assalariado = {
    '1': 'Sim, tem carteira de trabalho assinada',
    '2': 'Sim, é servidor público estatutário',
    '3': 'Não',
}
df['assalariado'] = df['assalariado'].astype(str)
df['assalariado'] = df['assalariado'].replace(assalariado)

In [22]:

# Faixa de rendimento
faixa = {
    '00': '0 - 100',
    '01': '101 - 300',
    '02': '301 - 600',
    '03': '601 - 800',
    '04': '801 - 1.600',
    '05': '1.601 - 3.000',
    '06': '3.001 - 10.000',
    '07': '10.001 - 50.000',
    '08': '50.001 - 100.000',
    '09': 'Mais de 100.000',
}
df['faixa_rendimento'] = df['faixa_rendimento'].astype(str)
df['faixa_rendimento'] = df['faixa_rendimento'].replace(faixa)

In [23]:
# Domicilio
domicilio = {
    '1': 'Próprio - já pago ',
    '2': 'Próprio - ainda pagando',
    '3': 'Alugado',
    '4': 'Cedido por empregador',
    '5': 'Cedido por familiar',
    '6': 'Cedido de outra forma',
    '7': 'Outra condição'
}
df['situacao_domicilio'] = df['situacao_domicilio'].astype(str)
df['situacao_domicilio'] = df['situacao_domicilio'].replace(domicilio)


print(df.head())

        uf  semana_mes  mes area_domicilio  idade    sexo cor_raca  \
0  Roraima           1    9         Urbana      1  Mulher    Parda   
1     Pará           1    9         Urbana      4  Mulher    Parda   
2    Ceará           1    9          Rural      4  Mulher    Parda   
3    Ceará           1    9         Urbana      2  Mulher   Branca   
4    Bahia           1    9         Urbana      5  Mulher    Parda   

    escolaridade     teve_febre teve_dificuldade_respirar  ...    teve_fadiga  \
0  Sem instrução  não aplicável             não aplicável  ...  não aplicável   
1  Sem instrução  não aplicável             não aplicável  ...  não aplicável   
2  Sem instrução  não aplicável             não aplicável  ...  não aplicável   
3  Sem instrução  não aplicável             não aplicável  ...  não aplicável   
4  Sem instrução  não aplicável             não aplicável  ...  não aplicável   

  teve_perda_cheiro foi_posto_saude  ficou_em_casa ficou_internado  \
0     não aplicável   

In [28]:
df.to_csv('output.csv', index=False)

from google.colab import files
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>